In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 5  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_5_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma5/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:1'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k_new = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)
            rel_err_out = (torch.norm(c_k - c_k_new) / torch.norm(c_k)).item() 
            sc = "{:0.1e}".format(rel_err_out)
            c_k = c_k_new
    
    return c_k

In [6]:
i = 0
psnrs_all = list()

with torch.no_grad():
    for runs in range(5):
        psnrs = list()
        for img in test_dataset:
            i = i + 1
            if True:
                gt = img.to(device)[None, :, :, :]
                noisy_image = (img.to(device) + (5/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
                denoised =safi_denoise(noisy_image)
                psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
                psnrs.append(psnr)
                print(i, psnr)
                
        psnrs_all.append(psnrs)

1 38.37
2 37.13
3 39.53
4 38.27
5 38.53
6 36.49
7 38.2
8 37.81
9 39.27
10 37.97
11 37.3
12 36.04
13 38.22
14 36.29
15 37.5
16 39.07
17 37.04
18 37.22
19 38.61
20 38.11
21 41.0
22 35.38
23 35.58
24 35.53
25 37.8
26 38.09
27 37.32
28 40.77
29 36.72
30 37.93
31 37.14
32 39.14
33 38.22
34 34.77
35 37.57
36 37.57
37 37.62
38 37.03
39 40.64
40 35.85
41 37.82
42 41.14
43 38.51
44 39.27
45 36.32
46 38.31
47 37.7
48 36.19
49 35.25
50 45.31
51 37.19
52 39.76
53 41.61
54 39.0
55 37.82
56 36.62
57 36.42
58 36.69
59 34.98
60 36.17
61 37.16
62 36.95
63 38.3
64 35.06
65 38.4
66 41.51
67 40.31
68 39.04
69 38.38
70 37.12
71 39.51
72 38.25
73 38.58
74 36.49
75 38.19
76 37.81
77 39.26
78 37.94
79 37.31
80 36.04
81 38.21
82 36.31
83 37.49
84 39.06
85 36.99
86 37.26
87 38.6
88 38.03
89 40.96
90 35.38
91 35.58
92 35.51
93 37.78
94 38.07
95 37.31
96 40.76
97 36.72
98 37.93
99 37.12
100 39.12
101 38.22
102 34.83
103 37.59
104 37.58
105 37.7
106 37.03
107 40.67
108 35.85
109 37.88
110 41.1
111 38.51
112 39.28


In [8]:
psnr_mat = np.zeros((5, 68))

for i in range(5):
    psnr_mat[i, :] = np.array(psnrs_all[i])

std_vec = np.std(psnr_mat, axis=0)
avg_vec = np.mean(psnr_mat, axis=0)

print('std mean:', np.mean(std_vec))
print('mean mean:', np.mean(avg_vec))

std mean: 0.015608193497092937
mean mean: 37.90555882352942
